***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import math
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 23 07:41 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
spark = SparkSession.builder.appName("YourAppName").config("spark.driver.maxResultSize", "8g").getOrCreate()
spark.conf.get("spark.driver.maxResultSize")

24/02/23 12:43:29 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


'8g'

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'irproject-414719bucket' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and not b.name.startswith("postings_gcp"):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb', 'became', 'may']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)
stemmer = PorterStemmer()

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

#####################################################################################################
stemDict = {}
l = ["genetics", "economic", "telecommunications", "computer", "bioinformatics"]
def word_count(text, id):
    containl = False
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    sizeOfDoc = 0
    word_counts = Counter()

    for token in tokens:
        if (token not in all_stopwords):
            if(token in stemDict):
                word_counts[token] += 1
            else:
                stemDict[token] = stemmer.stem(token)
                word_counts[stemDict[token]] += 1

            sizeOfDoc += 1
            if(containl == False):
                if(token in l):
                    containl = True

    if len(stemDict) >= 20000:
        stemDict.clear()

    if(containl == False):
        return []

    return [(token, (id, count, sizeOfDoc)) for token, count in word_counts.items()]

#####################################################################################################

def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key=lambda x: x[0])

#####################################################################################################

def calculate_df(postings):
    return postings.map(lambda x: (x[0], len(x[1])))

#####################################################################################################

def partition_postings_and_write(postings):

    bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), [(x[0], x[1])]))         
    grouped_postings = bucketed_postings.reduceByKey(lambda x, y: x + y)                        
    posting_locations = grouped_postings.map(lambda x: InvertedIndex.write_a_posting_list(x, 'postings_gcp', bucket_name))   

    return posting_locations

#####################################################################################################



In [ ]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_withNotSize = word_counts.map(lambda x: (x[0], (x[1][0], x[1][1])))

postings = word_counts_withNotSize.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>20)

w2df_dict = calculate_df(postings_filtered).collectAsMap()

_ = partition_postings_and_write(postings_filtered).collect()

super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)




In [ ]:
inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.df = w2df_dict
inverted.write_index('postings_gcp', 'index', bucket_name)

index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
Doc_TF = word_counts.map(lambda x: (x[1][0], (x[0], x[1][1]/x[1][2])))
Doc_TF = Doc_TF.groupByKey().mapValues(reduce_word_counts)

N = Doc_TF.count()

Doc_TF_IDF_filtered = Doc_TF.map(lambda x: (x[0], updated_value_function2(x[1])))
def updated_value_function2(post):
    listTermTfIdF = []

    for term, tf in post:
        if (term in w2df_dict):
            listTermTfIdF.append((term, tf * math.log10(N / w2df_dict[term])))

    nf = 0
    for term, tfIdf in listTermTfIdF:
        nf = nf + tfIdf ** 2
    nf = 1/math.sqrt(nf)

    return nf

Doc_TF_IDF_filtered_dict = Doc_TF_IDF_filtered.collectAsMap()

In [ ]:
inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.df = w2df_dict
inverted.nf = Doc_TF_IDF_filtered_dict
inverted.write_index('postings_gcp', 'index', bucket_name)

index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [14]:
inverted = InvertedIndex.read_index('postings_gcp', 'index', bucket_name)

In [15]:
# Create tf-idf dictionary
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
w2df_dict = inverted.df
N = len(inverted.df)
word_TF = word_counts.map(lambda x: (x[0], (x[1][0], x[1][1]/x[1][2])))
postings_TF = word_TF.groupByKey().mapValues(reduce_word_counts)
postings_TF_filtered = postings_TF.filter(lambda x: len(x[1])>20)

postings_TF_IDF_filtered = postings_TF_filtered.map(lambda x: (x[0], updated_value_function(x[0],x[1])))

def updated_value_function(term, post):
    listTermTfIdF = []
    
    if (term in w2df_dict):
        for doc, tf in post:
            listTermTfIdF.append((doc, tf * math.log10(N / w2df_dict[term])))
    
    return listTermTfIdF

postings_TF_IDF_filtered_dict = postings_TF_IDF_filtered.collectAsMap()

24/02/23 13:41:03 ERROR TaskSetManager: Total size of serialized results of 102 tasks (2.0 GiB) is bigger than spark.driver.maxResultSize (2.0 GiB)
24/02/23 13:41:03 WARN TaskSetManager: Lost task 101.0 in stage 6.0 (TID 411) (cluster-091d-w-0.c.irproject-414719.internal executor 9): TaskKilled (Tasks result size has exceeded maxResultSize)
24/02/23 13:41:03 WARN TaskSetManager: Lost task 106.0 in stage 6.0 (TID 416) (cluster-091d-w-2.c.irproject-414719.internal executor 4): TaskKilled (Stage cancelled)
24/02/23 13:41:03 WARN TaskSetManager: Lost task 105.0 in stage 6.0 (TID 415) (cluster-091d-w-1.c.irproject-414719.internal executor 12): TaskKilled (Stage cancelled)
24/02/23 13:41:03 WARN TaskSetManager: Lost task 104.0 in stage 6.0 (TID 414) (cluster-091d-w-1.c.irproject-414719.internal executor 11): TaskKilled (Stage cancelled)
24/02/23 13:41:03 WARN TaskSetManager: Lost task 103.0 in stage 6.0 (TID 413) (cluster-091d-w-2.c.irproject-414719.internal executor 8): TaskKilled (Stage ca

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 102 tasks (2.0 GiB) is bigger than spark.driver.maxResultSize (2.0 GiB)
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2717)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2653)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2652)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2652)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1189)
	at org.apache.spark.sched

Putting it all together

In [ ]:
inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.df = w2df_dict
inverted.tfidf = postings_TF_IDF_filtered_dict
inverted.nf = Doc_TF_IDF_filtered_dict
inverted.write_index('postings_gcp', 'index', bucket_name)

index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

In [ ]:
inverted_index = InvertedIndex.read_index('postings_gcp', 'index', bucket_name)

In [ ]:
simDoc = Counter()
query = "genetics"
query = dict(word_count(query, 0, True))
for term, value in query.items():
    query[term] = (value[0], value[1] * math.log10(N / w2df_dict[term]))

nfQuery = 0
for term, value in query.items():
    nfQuery = nfQuery + value[1] ** 2
nfQuery = 1/math.sqrt(nfQuery)


for term,  value in query.items():
    docTfItf = inverted.tfidf[term]
    for doc, weight in docTfItf:
        simDoc[doc] += value[1] * weight
        
for doc, sim in simDoc.items():
    simDoc[doc] = simDoc[doc] * nfQuery * inverted.nf[doc]

print(w2df_dict["genetics"])
top_10_similar_docs = simDoc.most_common(1000)

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
# Put your `generate_graph` function here

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 0 
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag